<a href="https://colab.research.google.com/github/ajmateti/ML-Projects/blob/master/medical_prediction_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data exploration and Preprocessing 

Load the data and build a Multi Class Classification

In [0]:
path = "/content/drive/My Drive/DS/df_diseases.csv"

In [0]:
import pandas as pd
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import tensorflow as tf
import numpy as np

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [0]:

len(stopwords.words("english"))

179

In [0]:
df = pd.read_csv(path,index_col=0)

In [0]:
df.head(10)

,name,link,symptoms,causes,risk_factor,overview,treatment,medication,home_remedies
0,Acanthosis nigricans,https://www.mayoclinic.org/diseases-conditions...,"[""Skin changes are the only signs of acanthosi...",['Acanthosis nigricans has been associated wit...,"['Acanthosis nigricans risk factors include:',...",['Acanthosis nigricans is a skin condition tha...,"['In many situations, treating the underlying ...",NaN,NaN
1,Achalasia,https://www.mayoclinic.org/diseases-conditions...,NaN,NaN,NaN,"[""Achalasia is a rare disorder that makes it d...",['Achalasia treatment focuses on relaxing or f...,NaN,NaN
2,Achilles tendon rupture,https://www.mayoclinic.org/diseases-conditions...,"[""Although it's possible to have no signs or s...",['Your Achilles tendon helps you point your fo...,['Factors that may increase your risk of Achil...,['The Achilles tendon is a strong fibrous cord...,['Treatment for a ruptured Achilles tendon oft...,NaN,NaN
3,Acute coronary syndrome,https://www.mayoclinic.org/diseases-conditions...,['The signs and symptoms of acute coronary syn...,['Acute coronary syndrome usually results from...,['The risk factors for acute coronary syndrome...,['Acute coronary syndrome is a term used to de...,['The immediate goals of treatment for acute c...,"['Depending on your diagnosis, medications for...",['Heart healthy lifestyle changes are an impor...
4,Adenomyosis,https://www.mayoclinic.org/diseases-conditions...,"['Sometimes, adenomyosis causes no signs or sy...","[""The cause of adenomyosis isn't known. There ...","['Risk factors for adenomyosis include:', 'Mos...","['With adenomyosis, the same tissue that lines...","['Adenomyosis often goes away after menopause,...",NaN,['To ease pelvic pain and cramping related to ...
5,Adjustment disorders,https://www.mayoclinic.org/diseases-conditions...,['Signs and symptoms depend on the type of adj...,['Adjustment disorders are caused by significa...,['Some things may make you more likely to have...,['Adjustment disorders are stress-related cond...,['Many people with adjustment disorders find t...,['Medications such as antidepressants and anti...,['Here are some steps you can take to care for...
6,Adnexal tumors,https://www.mayoclinic.org/diseases-conditions...,NaN,NaN,NaN,"['The ovaries, fallopian tubes, uterus, cervix...",NaN,NaN,NaN
7,Adrenal cancer,https://www.mayoclinic.org/diseases-conditions...,['Signs and symptoms of adrenal cancer include...,"[""It's not clear what causes adrenal cancer."",...",['Adrenal cancer happens more often in people ...,"['Perched atop each of your kidneys, your adre...","[""Adrenal cancer treatment usually involves su...",NaN,NaN
8,Adrenoleukodystrophy,https://www.mayoclinic.org/diseases-conditions...,NaN,NaN,NaN,['Adrenoleukodystrophy (uh-dree-noh-loo-koh-DI...,"['Adrenoleukodystrophy has no cure. However, s...",NaN,NaN
9,Adult Still's disease,https://www.mayoclinic.org/diseases-conditions...,"[""Most people with adult Still's disease have ...","[""It's not certain what causes adult Still's d...","[""Age is the main risk factor for adult Still'...","[""Adult Still's disease is a rare type of infl...","[""Doctors use a variety of drugs to treat adul...",NaN,"[""Here are ways to make the most of your healt..."


In [0]:
df["symptoms"].T[10]

'[\'Typical agoraphobia symptoms include fear of:\', "These situations cause anxiety because you fear you won\'t be able to escape or find help if you start to feel panicked or have other disabling or embarrassing symptoms.", \'In addition:\', "Some people have a panic disorder in addition to agoraphobia. Panic disorder is a type of anxiety disorder in which you experience sudden attacks of extreme fear that reach a peak within a few minutes and trigger intense physical symptoms (panic attacks). You might think that you\'re totally losing control, having a heart attack or even dying.", \'Fear of another panic attack can lead to avoiding similar circumstances or the place where it occurred in an attempt to prevent future panic attacks.\', \'Signs and symptoms of a panic attack can include:\', \'Agoraphobia can severely limit your ability to socialize, work, attend important events and even manage the details of daily life, such as running errands.\', "Don\'t let agoraphobia make your wo

In [0]:
# df = df[["symptoms","name"]]

In [0]:
len(df)

1182

In [0]:
# df = df[df[["name","symptoms"]].notna()]
df = df.dropna(subset=['name','symptoms'])
df["name"] = df["name"].astype(str)
df["symptoms"] = df["symptoms"].astype(str)
df["treatment"] = df["treatment"].astype(str)

In [0]:
len(df)

1098

In [0]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1098 entries, 0 to 1181
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   name           1098 non-null   object
 1   link           1098 non-null   object
 2   symptoms       1098 non-null   object
 3   causes         1080 non-null   object
 4   risk_factor    922 non-null    object
 5   overview       1098 non-null   object
 6   treatment      1098 non-null   object
 7   medication     290 non-null    object
 8   home_remedies  563 non-null    object
dtypes: object(9)
memory usage: 85.8+ KB


In [0]:
def preprocess(sentence,sw=None):
  sentence = sentence.lower().strip()
  split = sentence.split(".")
  if len(split)>11:
    split = split[-10:]
  sentence = ".".join(split)
  sentence = re.sub(r'[,?\"\[\]()]'," ",sentence)
  sentence = re.sub(r'[" "]+', " ", sentence)
  sentence = re.sub(r"[^a-z]+", " ", sentence)
  s=set(stopwords.words('english'))
  s.update(["symptoms","signs","may","include","doctor","usually","people","cause","see","might","often","one","medical","disease","common","make","also","seek","occur","time","vary","sometimes","type","notice","especially","within","even","person","including"])
  # print(sw)
  if sw is not None:
    s.update(sw)
  filtered_text = filter(lambda w: not w in s,sentence.split())
  return " ".join(filtered_text)


# def preprocess(sentence,sw):
#   sentence = sentence.lower().strip()
#   sentence = re.sub(r'[,?.\[\]()]'," \1 ",sentence)
#   sentence = " ".join(w.strip("'") for w in sentence.split(" ") if len(w)>1)
#   s=set(stopwords.words('english'))
#   s.update(["symptoms","signs","may","include","doctor","usually","people","cause","see","might","often","one","medical","disease","common","make","also","seek","occur","time","vary","sometimes","type","notice","especially","within","even","person","including"])
#   # print(sw)
#   if sw is not None:
#     s.update(sw)
#   filtered_text = filter(lambda w: not w in s,sentence.split())
#   sentence = " ".join(filtered_text)
#   if sentence.find("include:"):

#   return sentences

In [0]:
def count_sentences(series):
  sentence_count = []
  for sentence in series:
    sentence_count.append(len(sentence.split(".")))
  return pd.Series(sentence_count).value_counts()

In [0]:
count_sentences(df["symptoms"].astype(str))

4     128
3     100
6      99
8      94
5      90
7      81
2      64
9      56
10     45
11     36
12     28
1      27
13     25
14     24
15     20
17     19
18     19
16     15
19     15
20     14
21     11
23     10
22      9
26      5
25      5
27      5
31      5
30      4
24      4
32      4
34      4
42      3
39      3
38      2
44      2
41      2
55      2
29      2
49      2
48      1
54      1
56      1
52      1
51      1
50      1
28      1
46      1
45      1
43      1
65      1
40      1
37      1
33      1
96      1
dtype: int64

In [0]:
# df["symptoms"] = df["symptoms"].astype(str).map(preprocess)
for ind in df.index:
  df.loc[ind,"symptoms"] =  preprocess(df.loc[ind,"symptoms"],[df.loc[ind,"name"].lower()])
  # print(preprocess(df.loc[ind,"symptoms"],[df.loc[ind,"name"].lower()]))

In [0]:
# count_sentences(df["symptoms"].astype(str))

In [0]:
df["symptoms"].T[10]

'typical fear situations anxiety fear able escape find help start feel panicked disabling embarrassing addition panic disorder addition panic disorder anxiety disorder experience sudden attacks extreme fear reach peak minutes trigger intense physical panic attacks think totally losing control heart attack dying fear another panic attack lead avoiding similar circumstances place occurred attempt prevent future panic attacks panic attack severely limit ability socialize work attend important events manage details daily life running errands let world smaller call listed leaving home alone crowds waiting line enclosed spaces movie theaters elevators small stores open spaces parking lots bridges malls using public transportation bus plane train fear anxiety almost always results exposure situation fear anxiety proportion actual danger situation avoid situation need companion go endure situation extremely distressed experience significant distress problems social situations work areas life f

In [0]:
def count_unique_words(series):
  words_count = {}
  for sentence in series:
    words = sentence.split(" ")
    for word in words:
      word = word.strip()
      if word not in words_count:
        words_count[word] = 1
      else:
        words_count[word]+=1
  ret = {k: v for k, v in sorted(words_count.items(), key=lambda item: item[1],reverse=True)}
  count_df = pd.DataFrame(ret.items())
  return count_df

def count_words_per_sentence(series):
  sentence_count = []
  for sentence in series:
    sentence_count.append(len(sentence.split(" ")))
  return sentence_count

In [0]:
pd.set_option('display.expand_frame_repr', False)
count = count_unique_words(df["symptoms"].astype(str))
vocab_size = len(count)
print(vocab_size)
print(count[-4000:])

7691
                             0  1
3691                     bands  3
3692                   grating  3
3693                 avascular  3
3694                      ohss  3
3695                unexpected  3
...                        ... ..
7686                    stripe  1
7687                     wraps  1
7688  shinglesshinglesshingles  1
7689                      rich  1
7690                   rolling  1

[4000 rows x 2 columns]


In [0]:
count[count[1]>4]

,0,1
0,pain,1379
1,skin,710
2,blood,539
3,severe,504
4,loss,501
...,...,...
2710,booster,5
2711,chagas,5
2712,latent,5
2713,wart,5


In [0]:
max(count_words_per_sentence(df["symptoms"].astype(str)))

641

In [0]:
MAX_WORDS = 256

In [0]:
df.describe()

,name,link,symptoms,causes,risk_factor,overview,treatment,medication,home_remedies
count,1098,1098,1098,1080,922,1098,1098,290,563
unique,1098,1098,1098,1080,922,1098,1055,290,563
top,Heart failure,https://www.mayoclinic.org/diseases-conditions...,persistent depressive disorder come go period ...,['Atherosclerosis is a process in which blood ...,"[""Almost anyone can develop metatarsalgia, but...","[""Restless legs syndrome (RLS) is a condition ...",[],['Your doctor might recommend over-the-counter...,"['If you have a cramp, these actions may provi..."
freq,1,1,1,1,1,1,31,1,1


In [0]:
disease2id = {}
disease_count = 0
for disease in df["name"]:
  if disease not in disease2id:
    disease2id[disease] = disease_count 
    disease_count+=1
print(disease_count,disease2id)
id2disease = {v:k for k,v in disease2id.items()}
print(id2disease)
print(disease2id.keys())
diseases_list = list(disease2id.keys())

1098 {'Acanthosis nigricans': 0, 'Achilles tendon rupture': 1, 'Acute coronary syndrome': 2, 'Adenomyosis': 3, 'Adjustment disorders': 4, 'Adrenal cancer': 5, "Adult Still's disease": 6, 'Agoraphobia': 7, 'Alcohol intolerance': 8, 'Alcohol use disorder': 9, 'Alcoholic hepatitis': 10, 'Allergies': 11, 'Alpha-gal syndrome': 12, 'Ambiguous genitalia': 13, 'Ameloblastoma': 14, 'Amenorrhea': 15, 'Amnestic syndrome (See: Amnesia)': 16, 'Amniotic fluid embolism': 17, 'Ampullary cancer': 18, 'Amyloidosis': 19, 'Anal fissure': 20, 'Anaphylaxis': 21, 'Anemia': 22, 'Aneurysm, abdominal aortic (See: Abdominal aortic aneurysm)': 23, 'Angelman syndrome': 24, 'Angiosarcoma': 25, 'Ankylosing spondylitis': 26, 'Anorgasmia in women': 27, 'Anterior cruciate ligament injury (See: ACL injury)': 28, 'Anthrax': 29, 'Antiphospholipid syndrome': 30, 'Anxiety disorders': 31, 'Aortic valve disease': 32, 'Aphasia': 33, 'Aplastic anemia': 34, 'Appendicitis': 35, 'ARDS': 36, 'Arteriovenous fistula': 37, 'Arterioven

In [0]:
#Only consider symptoms which is a part of atleast 4 diseases
symptoms = count[count[1]>4][0]
symptom2id = {}
id2symptom = {}
for ind in symptoms.index:
  if len(symptoms.iloc[ind])>2:
    symptom2id[symptoms.iloc[ind]] = ind
    id2symptom[ind] = symptoms.iloc[ind]
print(len(symptom2id),symptom2id)
print(id2symptom)


2694 {'pain': 0, 'skin': 1, 'blood': 2, 'severe': 3, 'loss': 4, 'child': 5, 'fever': 6, 'problems': 7, 'difficulty': 8, 'heart': 9, 'swelling': 10, 'body': 11, 'care': 13, 'experience': 14, 'chest': 15, 'breath': 16, 'develop': 17, 'vomiting': 18, 'vision': 19, 'fatigue': 20, 'weakness': 21, 'eye': 22, 'changes': 23, 'treatment': 24, 'weight': 25, 'infection': 26, 'condition': 27, 'pressure': 28, 'emergency': 29, 'abdominal': 30, 'nausea': 31, 'eyes': 32, 'breathing': 33, 'shortness': 34, 'red': 35, 'muscle': 36, 'appointment': 37, 'feeling': 38, 'syndrome': 39, 'headache': 40, 'back': 41, 'days': 42, 'persistent': 43, 'affected': 44, 'rash': 45, 'neck': 46, 'disorder': 47, 'bleeding': 48, 'appear': 49, 'diarrhea': 50, 'children': 51, 'high': 52, 'call': 53, 'mouth': 54, 'help': 55, 'area': 56, 'mild': 57, 'first': 58, 'sleep': 59, 'small': 60, 'cancer': 61, 'legs': 62, 'causes': 63, 'sudden': 64, 'feel': 65, 'get': 66, 'painful': 67, 'urine': 68, 'throat': 69, 'many': 70, 'abdomen': 7

In [0]:
column_names = list(symptom2id.keys())
column_names.extend(["Disease"])
print(column_names)

['pain', 'skin', 'blood', 'severe', 'loss', 'child', 'fever', 'problems', 'difficulty', 'heart', 'swelling', 'body', 'care', 'experience', 'chest', 'breath', 'develop', 'vomiting', 'vision', 'fatigue', 'weakness', 'eye', 'changes', 'treatment', 'weight', 'infection', 'condition', 'pressure', 'emergency', 'abdominal', 'nausea', 'eyes', 'breathing', 'shortness', 'red', 'muscle', 'appointment', 'feeling', 'syndrome', 'headache', 'back', 'days', 'persistent', 'affected', 'rash', 'neck', 'disorder', 'bleeding', 'appear', 'diarrhea', 'children', 'high', 'call', 'mouth', 'help', 'area', 'mild', 'first', 'sleep', 'small', 'cancer', 'legs', 'causes', 'sudden', 'feel', 'get', 'painful', 'urine', 'throat', 'many', 'abdomen', 'baby', 'breast', 'head', 'feet', 'eating', 'serious', 'side', 'two', 'occurs', 'swollen', 'frequent', 'away', 'early', 'attention', 'rapid', 'health', 'joint', 'headaches', 'others', 'talk', 'face', 'need', 'weeks', 'trouble', 'contact', 'life', 'nose', 'without', 'right', '

In [0]:
new_df = pd.DataFrame(columns=column_names)
new_df["Disease"] = list(disease2id.keys())
new_df.set_index("Disease",inplace=True)
new_df.fillna(0,inplace=True)
new_df

,pain,skin,blood,severe,loss,child,fever,problems,difficulty,heart,swelling,body,care,experience,chest,breath,develop,vomiting,vision,fatigue,weakness,eye,changes,treatment,weight,infection,condition,pressure,emergency,abdominal,nausea,eyes,breathing,shortness,red,muscle,appointment,feeling,syndrome,headache,...,estrogen,dialysis,circulation,positive,narcissistic,schizotypal,belief,frontotemporal,als,sti,covers,placental,mercury,miliaria,aldosteronism,atresia,reactive,reye,chickenpox,rheumatic,rocky,salmonella,schizoaffective,spring,sheehan,msa,narcolepsy,soy,bees,stung,bee,compressions,supraventricular,tapeworm,patellar,booster,chagas,latent,wart,triple
Disease,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Acanthosis nigricans,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Achilles tendon rupture,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Acute coronary syndrome,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Adenomyosis,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Adjustment disorders,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yips,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Zika virus,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Zollinger-Ellison syndrome,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
for ind in df.index:
  symptoms_list = list(df.loc[ind,"symptoms"].lower().split(" "))
  for symptom in symptoms_list:
    if symptom in new_df.columns:
      new_df.loc[df.loc[ind,"name"],symptom] = 1
new_df

,pain,skin,blood,severe,loss,child,fever,problems,difficulty,heart,swelling,body,care,experience,chest,breath,develop,vomiting,vision,fatigue,weakness,eye,changes,treatment,weight,infection,condition,pressure,emergency,abdominal,nausea,eyes,breathing,shortness,red,muscle,appointment,feeling,syndrome,headache,...,estrogen,dialysis,circulation,positive,narcissistic,schizotypal,belief,frontotemporal,als,sti,covers,placental,mercury,miliaria,aldosteronism,atresia,reactive,reye,chickenpox,rheumatic,rocky,salmonella,schizoaffective,spring,sheehan,msa,narcolepsy,soy,bees,stung,bee,compressions,supraventricular,tapeworm,patellar,booster,chagas,latent,wart,triple
Disease,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Acanthosis nigricans,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Achilles tendon rupture,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Acute coronary syndrome,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,1,0,1,0,0,0,0,0,0,0,1,1,0,1,0,0,1,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Adenomyosis,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Adjustment disorders,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yips,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Zika virus,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Zollinger-Ellison syndrome,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
def get_disease_id(name):
  return disease2id[name]

In [0]:
new_df.index.map(get_disease_id)

Int64Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
            ...
            1088, 1089, 1090, 1091, 1092, 1093, 1094, 1095, 1096, 1097],
           dtype='int64', name='Disease', length=1098)

In [0]:
X = new_df[list(symptom2id.keys())].to_numpy()
y = new_df.index.map(get_disease_id).to_numpy()
print(X.shape,y.shape)
print(X[0])

(1098, 2694) (1098,)
[0 1 0 ... 0 0 0]


In [0]:
from tensorflow.keras.utils import to_categorical

In [0]:
y_cat = to_categorical(y)
y_cat.shape

(1098, 1098)

In [0]:
X = np.array(X)

In [0]:
y = np.array(y)

In [0]:
import tensorflow as tf

In [0]:
X_tensor = tf.data.Dataset.from_tensor_slices(X)

In [0]:
y_tensor = tf.data.Dataset.from_tensor_slices(y)

In [0]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(1024,activation="relu"))
model.add(tf.keras.layers.Dropout(0.7))
model.add(tf.keras.layers.Dense(512,activation="relu"))
model.add(tf.keras.layers.Dropout(0.7))
model.add(tf.keras.layers.Dense(64,activation="relu"))
model.add(tf.keras.layers.Dropout(0.7))
model.add(tf.keras.layers.Dense(y_cat.shape[0],activation="sigmoid"))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [0]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(1024,activation="relu"))
model.add(tf.keras.layers.Dropout(0.7))
model.add(tf.keras.layers.Dense(256,activation="relu"))
model.add(tf.keras.layers.Dropout(0.7))
model.add(tf.keras.layers.Dense(y_cat.shape[0],activation="sigmoid"))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [0]:
model.fit(X,y_cat,epochs=90)

Epoch 1/90
35/35 [==============================] - 0s 4ms/step - loss: 7.0329 - accuracy: 0.0000e+00
Epoch 2/90
35/35 [==============================] - 0s 4ms/step - loss: 6.9685 - accuracy: 0.0036
Epoch 3/90
35/35 [==============================] - 0s 4ms/step - loss: 6.8735 - accuracy: 0.0091
Epoch 4/90
35/35 [==============================] - 0s 4ms/step - loss: 6.7201 - accuracy: 0.0137
Epoch 5/90
35/35 [==============================] - 0s 4ms/step - loss: 6.4637 - accuracy: 0.0392
Epoch 6/90
35/35 [==============================] - 0s 4ms/step - loss: 6.1726 - accuracy: 0.0556
Epoch 7/90
35/35 [==============================] - 0s 4ms/step - loss: 5.8138 - accuracy: 0.0883
Epoch 8/90
35/35 [==============================] - 0s 4ms/step - loss: 5.4661 - accuracy: 0.1066
Epoch 9/90
35/35 [==============================] - 0s 4ms/step - loss: 5.0815 - accuracy: 0.1457
Epoch 10/90
35/35 [==============================] - 0s 4ms/step - loss: 4.8252 - accuracy: 0.1776
Epoch 11/90
35/

In [0]:
model.save("/content/drive/My Drive/DS/medical_chatbot/medical-chatbot2-using-excel")

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: /content/drive/My Drive/DS/medical_chatbot/medical-chatbot2-using-excel/assets


In [0]:
import difflib
def get_new_x(my_input):
  my_input = preprocess(my_input,None)
  new_x = [0 for i in range(len(symptom2id))]
  known_symptoms = 0
  for word in my_input.split():
    new_symptoms = difflib.get_close_matches(word,symptom2id.keys(),cutoff=0.85)
    print(new_symptoms)
    for new_symptom in new_symptoms:
      new_x[symptom2id[new_symptom]] += 1
      known_symptoms+=1
  return new_x

In [0]:
text = input()

i have elevated temperature.I have Sweating, 'Chills and shivering', 'Headache', 'Muscle aches', 'Loss of appetite', 'Irritability', 'Dehydration'


In [0]:
print(get_new_x(text))

['elevated']
['temperature']
['sweating', 'eating']
['chills', 'achilles']
['shivering']
['headache', 'headaches']
['muscle', 'muscles']
['aches', 'ache']
['loss', 'loses']
['appetite']
['irritability']
['dehydration']
[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [0]:
print(np.max(model.predict_proba([get_new_x(text)])))

['elevated']
['temperature']
['sweating', 'eating']
['chills', 'achilles']
['shivering']
['headache', 'headaches']
['muscle', 'muscles']
['aches', 'ache']
['loss', 'loses']
['appetite']
['irritability']
['dehydration']
Instructions for updating:
Please use `model.predict()` instead.
0.0058238152


In [0]:
print(model.predict([get_new_x(text)]).argsort()[0][-20:][::-1])

['elevated']
['temperature']
['sweating', 'eating']
['chills', 'achilles']
['shivering']
['headache', 'headaches']
['muscle', 'muscles']
['aches', 'ache']
['loss', 'loses']
['appetite']
['irritability']
['dehydration']
[ 594  830 1093  319   28  646  777  311  508  555  161  739  420 1091
  247   91  556  131  711  258]


In [0]:
print(model.predict([get_new_x(text)])[0][238])

In [0]:
print("Treatment: \n {}".format(df.loc[model.predict_classes([get_new_x(text)])[0],"treatment"]))

['elevated']
['temperature']
['sweating', 'eating']
['chills', 'achilles']
['shivering']
['headache', 'headaches']
['muscle', 'muscles']
['aches', 'ache']
['loss', 'loses']
['appetite']
['irritability']
['dehydration']
Treatment: 
 ["Treatment for childhood obesity is based on your child's age and if he or she has other medical conditions. Treatment usually includes changes in your child's eating habits and physical activity level. In certain circumstances, treatment might include medications or weight-loss surgery."]
